要学习backtrader源码，首先要具备的基础就是元类知识，在python-元类基础已经对基本知识进行了说明。metabase 文件是backtrader的基础，逐一看每个函数，进行用例测试。

##### 1  findbases函数

In [4]:
# findbases函数：递归找给定类到某个基类之间的全部类
def findbases(kls, topclass):
    retval = list()
    for base in kls.__bases__:
       # print(base)
        if issubclass(base, topclass):
            retval.extend(findbases(base, topclass))
            retval.append(base)

    return retval

知识点与用例：

issubclass() 是python内置函数，它用于检查一个类是否继承自另一个类；
__bases__ 是一个类的特殊属性，它包含了该类的所有直接父类（或称为基类）；

In [5]:
# 1.1 findbases的例子说明
class Grandparent:  
    pass  
  
class Parent1(Grandparent):  
    pass  
  
class Parent2:  
    pass  
  
class Child(Parent1, Parent2):  
    pass  
  
# 检查 Child 是否是 Grandparent 的子类  
print(issubclass(Child, Grandparent))  # 输出: True  
  
# 检查 Child 是否是 Parent1 的子类  
print(issubclass(Child, Parent1))  # 输出: True  
  
# 检查 Child 是否是 Parent2 的子类  
print(issubclass(Child, Parent2))  # 输出: True  
  
# 检查 Child 是否是其自身的子类  
print(issubclass(Child, Child))  # 输出: True  
  
# 检查 Parent1 是否是 Child 的子类（不是）  
print(issubclass(Parent1, Child))  # 输出: False  
  
# 检查 Grandparent 是否是 Parent1 的子类（不是）  
print(issubclass(Grandparent, Parent1))  # 输出: False
  
# 检查 Parent2 是否是 Grandparent 的子类（不是）  
print(issubclass(Parent2, Grandparent))  # 输出: False

# 打印Child的基类
print(Child.__bases__)  # 输出 Parent1，Parent2

# 找Child的全部父类
print(findbases(Child, Grandparent)) # 输出Grandparent，Parent1
# 找Child和type之间全部类
print(findbases(Child, object)) # 输出object、Grandparent'、Parent1'、object、Parent2

True
True
True
True
False
False
False
(<class '__main__.Parent1'>, <class '__main__.Parent2'>)
[<class '__main__.Grandparent'>, <class '__main__.Parent1'>]
[<class 'object'>, <class '__main__.Grandparent'>, <class '__main__.Parent1'>, <class 'object'>, <class '__main__.Parent2'>]


##### 2 findowner 函数

In [6]:
# 遍历堆栈，寻找不等于owned 和 skip 的 cls的对象
# 

def findowner(owned, cls, startlevel=2, skip=None):
    # skip this frame and the caller's -> start at 2
    for framelevel in itertools.count(startlevel):
        try:
            frame = sys._getframe(framelevel)
        except ValueError:
            # Frame depth exceeded ... no owner ... break away
            break

        # 'self' in regular code
        self_ = frame.f_locals.get('self', None)
        if skip is not self_:
            if self_ is not owned and isinstance(self_, cls):
                return self_

        # '_obj' in metaclasses
        obj_ = frame.f_locals.get('_obj', None)
        if skip is not obj_:
            if obj_ is not owned and isinstance(obj_, cls):
                return obj_

    return None

知识点与用例测试：

In [9]:
# 2.1 itertools.count 返回一个无限迭代器，从给定的起始值开始，每次递增一个固定的步长
# itertools.count(start=0, step=1)
# - start：计数器的起始值，默认为 0。
# - step：每次迭代的步长，默认为 1
import itertools  
for framelevel in itertools.count(2):
    print(framelevel)
    if  framelevel > 3 :
        break

2
3
4


In [13]:
# 2.2 sys._getframe() 用于获取当前调用栈中的帧对象。帧对象代表了执行上下文，包含了函数参数、局部变量等信息。
# 接受一个可选的参数 depth，表示要获取的帧的深度。
# 默认情况下，depth 为 0，表示获取当前帧。如果 depth 为正数，那么函数将返回调用栈中 depth 深度处的帧对象。
# 如果 depth 为负数，那么函数将返回调用栈中从当前帧开始往上数 abs(depth) 帧处的帧对象。

# f_locals 获取当前帧（或指定深度的帧）局部变量的方法。f_locals 是一个字典，它包含了在该帧中定义的局部变量。


def outer_function():  
    outer_param1 = "outer"
    print("Outer function called")  
    middle_function()  

def middle_function():
    middle_param1= "middle"
    print("Middle function called")  
    inner_function()  
    
def inner_function():  
    inner_param1= "inner"
    print("Inner function called")  
    frame = sys._getframe(2)  # 获取调用链中第2层的帧对象  
    print("Second frame f_locals:",frame.f_locals) # out:Second frame f_locals: {'outer_param1': 'outer'}
    print("Second frame function name:", frame.f_code.co_name)  # out:Second frame function name: outer_function
    print("Second frame line number:", frame.f_lineno)  # out: Second frame line number: 12
    print("-----------------------------------------")
    frame = sys._getframe(1)  # 获取调用链中第2层的帧对象  
    print("First frame f_locals:",frame.f_locals) # out: First frame f_locals: {'middle_param1': 'middle'}
    print("First frame function name:", frame.f_code.co_name)  # out: First frame function name: middle_function
    print("First frame line number:", frame.f_lineno) # out: First frame line number: 17

# 调用最外层的函
outer_function()

Outer function called
Middle function called
Inner function called
Second frame f_locals: {'outer_param1': 'outer'}
Second frame function name: outer_function
Second frame line number: 12
First frame f_locals: {'middle_param1': 'middle'}
First frame function name: middle_function
First frame line number: 17


In [ ]:
# 3.3 MetaBase,这个是类的基本声明

In [15]:
# 定义MetaBase类
class MetaBase(type):
    # donew 执行之前调用的函数
    def doprenew(cls, *args, **kwargs):
        return cls, args, kwargs

    def donew(cls, *args, **kwargs):
        _obj = cls.__new__(cls, *args, **kwargs)
        return _obj, args, kwargs

    # doinit 之前调用的函数
    def dopreinit(cls, _obj, *args, **kwargs):
        return _obj, args, kwargs

    def doinit(cls, _obj, *args, **kwargs):
        _obj.__init__(*args, **kwargs)
        return _obj, args, kwargs

    # doinit 执行后调用的函数
    def dopostinit(cls, _obj, *args, **kwargs):
        return _obj, args, kwargs

    # 在 Python 中，__call__ 是一个特殊的方法，它允许一个对象像函数那样被调用。
    # 如果一个类定义了 __call__ 方法，那么它的实例就可以像函数那样被调用
    def __call__(cls, *args, **kwargs):
        cls, args, kwargs = cls.doprenew(*args, **kwargs)
        _obj, args, kwargs = cls.donew(*args, **kwargs)
        _obj, args, kwargs = cls.dopreinit(_obj, *args, **kwargs)
        _obj, args, kwargs = cls.doinit(_obj, *args, **kwargs)
        _obj, args, kwargs = cls.dopostinit(_obj, *args, **kwargs)
        return _obj